In [132]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [133]:
# load processes features
feature_df = pd.read_csv('model/train/output/feature_set/feature_set.csv')
feature_df = feature_df.drop(columns=['Unnamed: 0'])

In [134]:
# save label encoding mapping
label_encoder_map = {label:i for i, label in enumerate(feature_df.label.unique())}

# encode labels
feature_df['encoded_label'] = feature_df['label'].map(label_encoder_map)
y = feature_df['label'].values.ravel()
y_encoded = feature_df['encoded_label'].values.ravel()


In [135]:
# scale numeric/all features
def scale_features(feature_set:pd.DataFrame)->pd.DataFrame:
    scaler = StandardScaler()
    feature_set = pd.DataFrame(scaler.fit_transform(feature_set), columns=feature_set.columns, index=feature_set.index)
    return feature_set

X = feature_df[[col for col in feature_df.columns if not col.endswith('label')]]
# X = scale_features(X)



In [136]:
RANDOM_STATE = 42
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=.2, stratify=y, shuffle=True, random_state=RANDOM_STATE)
Xe_train, Xe_test, ye_train, ye_test =  train_test_split(X, y_encoded, test_size=.2, stratify=y, shuffle=True, random_state=RANDOM_STATE)

In [97]:
RANDOM_STATE = 42
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=.2, stratify=y, shuffle=True, random_state=RANDOM_STATE)

In [137]:

pipeline = Pipeline([
            ('clf', RandomForestClassifier(random_state=0))
        ])

param_grid = [
            {'clf__n_estimators': [100, 500, 1000],
             'clf__criterion': ['gini', 'entropy'],
             'clf__max_depth': [10, 100, 1000]}
        ]

estimator = GridSearchCV(pipeline, param_grid, cv=10, scoring='accuracy')

model = estimator.fit(X_train, y_train)

y_pred = model.predict(X_test)

perf_rf = {'accuracy': accuracy_score(y_test, y_pred),
                'recall': recall_score(y_test, y_pred, average='micro'),
                'precision': precision_score(y_test, y_pred, average='micro'),
                'f1': f1_score(y_test, y_pred, average='micro')}



In [139]:
perf_rf

{'accuracy': 0.7115384615384616,
 'recall': 0.7115384615384616,
 'precision': 0.7115384615384616,
 'f1': 0.7115384615384616}

In [140]:
# xgboost

import xgboost as xgb
xgbr = xgb.XGBClassifier(objective='multi:softprob')
xgbr.fit(Xe_train, ye_train)
 
y_pred_xgb = xgbr.predict(Xe_test)
perf_xgb = {'accuracy': accuracy_score(ye_test, y_pred_xgb),
                'recall': recall_score(ye_test, y_pred_xgb, average='micro'),
                'precision': precision_score(ye_test, y_pred_xgb, average='micro'),
                'f1': f1_score(ye_test, y_pred_xgb, average='micro')}

In [141]:
perf_xgb

{'accuracy': 0.6538461538461539,
 'recall': 0.6538461538461539,
 'precision': 0.6538461538461539,
 'f1': 0.6538461538461539}

In [1]:
from pycaret.classification import *
s = setup(feature_df, target='label')

OSError: dlopen(/opt/anaconda3/envs/moo_dev/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /opt/anaconda3/envs/moo_dev/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)